## Preparing Dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = 'Rice Leaf Diseases'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                 target_size=(300,300),
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                 target_size=(300,300),
                                                 batch_size=64,
                                                 class_mode='categorical',
                                                 shuffle=True)

FileNotFoundError: ignored

In [ ]:
train_generator.class_indices

{'Bacterialblight': 0, 'Blast': 1, 'Brownspot': 2, 'Tungro': 3}

In [ ]:
#batches
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)

Image batch shape:  (64, 300, 300, 3)
Label batch shape:  (64, 4)


In [ ]:
#label
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

['Bacterialblight' 'Blast' 'Brownspot' 'Tungro']


## Setup Pre-Trained Resnet50

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adagrad, Adam
from tensorflow.keras.applications import ResNet50



In [ ]:
pretrained_model=ResNet50(
    include_top=False,
    pooling='avg', classes=4,
    weights= 'imagenet',
    input_shape=(300,300,3))

for layer in pretrained_model.layers :
    layer.trainable = False

pretrained_model.summary()

94765736/94765736 [==============================] - 187s 2us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 306, 306, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                        

 conv2_block3_1_bn (BatchNormal  (None, 75, 75, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 75, 75, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 75, 75, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 75, 75, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
          

 conv3_block3_1_bn (BatchNormal  (None, 38, 38, 128)  512        ['conv3_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 38, 38, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 38, 38, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 38, 38, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block2_1_conv (Conv2D)   (None, 19, 19, 256)  262400      ['conv4_block1_out[0][0]']       
                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 19, 19, 256)  590080      ['conv4_block2_1_relu[0][0]']    
          

                                )                                                                 
                                                                                                  
 conv4_block5_1_conv (Conv2D)   (None, 19, 19, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_blo

 conv5_block1_3_bn (BatchNormal  (None, 10, 10, 2048  8192       ['conv5_block1_3_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 conv5_block1_add (Add)         (None, 10, 10, 2048  0           ['conv5_block1_0_bn[0][0]',      
                                )                                 'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 10, 10, 2048  0           ['conv5_block1_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 10, 10, 512)  1049088     ['conv5_block1_out[0][0]']       
          

In [ ]:
resnet_model = Sequential()
resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(Dropout(0.2))
resnet_model.add(Dense(4, activation='softmax')
) 

In [ ]:
resnet_model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
history = resnet_model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs=150,
                    batch_size=64)

# Model Evaluation

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
plt.plot(epochs, loss, 'r', label='Training accuracy')
plt.plot(epochs, val_loss, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

# Save Model

In [ ]:
resnet_model.save('model.h5')

In [ ]:
from keras.models import model_from_json

json_model = history.to_json()

with open('resnet_model.json', 'w') as json_file:
    json_file.write(json_model)

## Testing Model

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 23,850,500
Trainable params: 262,788
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import img_to_array

x = load_img(
    'test.jpg',
    grayscale=False,
    color_mode='rgb',
    target_size=[300,300],
    interpolation='nearest',
)
x = img_to_array(x)
x = x/255.0
x = np.array([x])

In [ ]:
model.predict(x)

1/1 [==============================] - 2s 2s/step


array([[0.11570001, 0.46742785, 0.40599984, 0.0108724 ]], dtype=float32)

## Convert to TFLite

In [ ]:
converter = tensorflow.lite.TFLiteConverter.from_keras_model(model) #convert to tflite
converter.optimizations = [tensorflow.lite.Optimize.DEFAULT] #post training quantization

In [ ]:
tflite_model = converter.convert()
tflite_model_file = 'model.tflite'

with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\JIHANN~1\AppData\Local\Temp\tmpvzw3ysn7\assets


INFO:tensorflow:Assets written to: C:\Users\JIHANN~1\AppData\Local\Temp\tmpvzw3ysn7\assets


In [ ]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
 f.write(labels)

## Test TFlite

In [ ]:
# Load TFLite model and allocate tensors.

interpreter = tensorflow.lite.Interpreter(model_path=tflite_model_file)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
input_details

[{'name': 'serving_default_resnet50_input:0',
  'index': 0,
  'shape': array([  1, 300, 300,   3]),
  'shape_signature': array([ -1, 300, 300,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 191,
  'shape': array([1, 4]),
  'shape_signature': array([-1,  4]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
input_shape = input_details[0]['shape']
interpreter.set_tensor(input_details[0]['index'], x)

interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.11569633 0.46742406 0.40600696 0.01087269]]
